In [ ]:
from flask import Flask, request, jsonify, send_file, redirect, url_for, render_template_string, session, flash
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from flask_caching import Cache
from flask_wtf.csrf import CSRFProtect
from werkzeug.security import generate_password_hash, check_password_hash
import os, csv, tempfile, threading, time, statistics, io, json, secrets, smtplib, qrcode
from typing import List, Dict, Optional, Tuple
from datetime import datetime, timedelta
from dataclasses import dataclass
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
import jwt
import logging
from logging.handlers import RotatingFileHandler, SMTPHandler
import redis
from sqlalchemy import create_engine, Column, String, Integer, Float, DateTime, Text, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import asyncio
import aiofiles
from concurrent.futures import ThreadPoolExecutor
import boto3
from botocore.exceptions import ClientError
from prometheus_client import Counter, Histogram, generate_latest, CONTENT_TYPE_LATEST
import docker
from kubernetes import client, config

class Config:
    SECRET_KEY = os.environ.get('SECRET_KEY', 'dev-secret-key-change-in-production')
    ADMIN_TOKEN = os.environ.get('ADMIN_TOKEN', 'hackathon_secret_token_123')
    DATABASE_URL = os.environ.get('DATABASE_URL', 'sqlite:///students.db')
    REDIS_URL = os.environ.get('REDIS_URL', 'redis://localhost:6379')
    SMTP_SERVER = os.environ.get('SMTP_SERVER', 'smtp.gmail.com')
    SMTP_PORT = int(os.environ.get('SMTP_PORT', 587))
    SMTP_USER = os.environ.get('SMTP_USER', '')
    SMTP_PASSWORD = os.environ.get('SMTP_PASSWORD', '')
    AWS_ACCESS_KEY = os.environ.get('AWS_ACCESS_KEY', '')
    AWS_SECRET_KEY = os.environ.get('AWS_SECRET_KEY', '')
    S3_BUCKET = os.environ.get('S3_BUCKET', 'student-management-backups')
    ENABLE_K8S = os.environ.get('ENABLE_K8S', 'false').lower() == 'true'
    BACKUP_INTERVAL = int(os.environ.get('BACKUP_INTERVAL', 3600))  # 1 hour

Base = declarative_base()

class Student(Base):
    __tablename__ = 'students'
    id = Column(String(20), primary_key=True)
    name = Column(String(100), nullable=False)
    age = Column(Integer, nullable=False)
    grade = Column(String(1), nullable=False)
    marks = Column(Integer, nullable=False)
    email = Column(String(100))
    phone = Column(String(20))
    address = Column(Text)
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    is_active = Column(Boolean, default=True)

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True, nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    password_hash = Column(String(255), nullable=False)
    role = Column(String(20), default='user')  # admin, teacher, user
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)

class AuditLog(Base):
    __tablename__ = 'audit_logs'
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer)
    action = Column(String(50), nullable=False)
    resource = Column(String(100))
    details = Column(Text)
    ip_address = Column(String(45))
    user_agent = Column(Text)
    timestamp = Column(DateTime, default=datetime.utcnow)

# ===== Advanced Analytics Engine =====
class AdvancedAnalytics:
    def __init__(self, students_data):
        self.df = pd.DataFrame(students_data)
        self.results = {}
    
    def perform_comprehensive_analysis(self):
        if self.df.empty:
            return {"error": "No data available"}
        
        self._basic_statistics()
        self._trend_analysis()
        self._correlation_analysis()
        self._predictive_insights()
        self._anomaly_detection()
        self._cluster_analysis()
        
        return self.results
    
    def _basic_statistics(self):
        self.results['basic'] = {
            'count': len(self.df),
            'marks_stats': {
                'mean': self.df['marks'].mean(),
                'median': self.df['marks'].median(),
                'std': self.df['marks'].std(),
                'min': self.df['marks'].min(),
                'max': self.df['marks'].max(),
                'skewness': self.df['marks'].skew(),
                'kurtosis': self.df['marks'].kurtosis()
            },
            'age_stats': {
                'mean': self.df['age'].mean(),
                'std': self.df['age'].std()
            }
        }
    
    def _trend_analysis(self):
        age_groups = pd.cut(self.df['age'], bins=3)
        trend = self.df.groupby(age_groups)['marks'].mean()
        self.results['trends'] = {
            'age_marks_trend': trend.to_dict(),
            'performance_trend': 'improving' if trend.iloc[-1] > trend.iloc[0] else 'declining'
        }
    
    def _correlation_analysis(self):
        correlation = self.df[['age', 'marks']].corr().iloc[0,1]
        self.results['correlations'] = {
            'age_marks_correlation': correlation,
            'strength': 'strong' if abs(correlation) > 0.7 else 'moderate' if abs(correlation) > 0.3 else 'weak'
        }
    
    def _predictive_insights(self):
        if len(self.df) > 1:
            slope, intercept, r_value, p_value, std_err = stats.linregress(
                self.df['age'], self.df['marks']
            )
            self.results['predictive'] = {
                'regression_slope': slope,
                'r_squared': r_value**2,
                'prediction_confidence': 'high' if r_value**2 > 0.6 else 'medium' if r_value**2 > 0.3 else 'low'
            }
    
    def _anomaly_detection(self):
        z_scores = np.abs(stats.zscore(self.df['marks']))
        anomalies = self.df[z_scores > 2]
        self.results['anomalies'] = {
            'count': len(anomalies),
            'students': anomalies.to_dict('records')
        }
    
    def _cluster_analysis(self):
        # Simple clustering based on age and marks
        from sklearn.cluster import KMeans
        if len(self.df) >= 3:
            X = self.df[['age', 'marks']].values
            kmeans = KMeans(n_clusters=min(3, len(self.df)), random_state=42)
            clusters = kmeans.fit_predict(X)
            self.df['cluster'] = clusters
            self.results['clusters'] = {
                'cluster_centers': kmeans.cluster_centers_.tolist(),
                'students_per_cluster': pd.Series(clusters).value_counts().to_dict()
            }

class PerformancePredictor:
    def __init__(self):
        self.model = None
        self.is_trained = False
    
    def train(self, students_data):
        if len(students_data) < 10:
            return False
        
        df = pd.DataFrame(students_data)
        X = df[['age', 'previous_marks']].fillna(0)  # Assuming we have previous marks
        y = df['marks']
        
        from sklearn.ensemble import RandomForestRegressor
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.model.fit(X, y)
        self.is_trained = True
        return True
    
    def predict(self, age, previous_marks):
        if not self.is_trained:
            return None
        return self.model.predict([[age, previous_marks]])[0]

app = Flask(__name__)
app.config.from_object(Config)

limiter = Limiter(app, key_func=get_remote_address)
cache = Cache(app, config={'CACHE_TYPE': 'redis', 'CACHE_REDIS_URL': Config.REDIS_URL})
csrf = CSRFProtect(app)

engine = create_engine(Config.DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base.metadata.create_all(bind=engine)

redis_client = redis.from_url(Config.REDIS_URL)

s3_client = boto3.client(
    's3',
    aws_access_key_id=Config.AWS_ACCESS_KEY,
    aws_secret_access_key=Config.AWS_SECRET_KEY
) if Config.AWS_ACCESS_KEY else None

REQUEST_COUNT = Counter('http_requests_total', 'Total HTTP Requests', ['method', 'endpoint', 'status'])
REQUEST_DURATION = Histogram('http_request_duration_seconds', 'HTTP Request Duration')

thread_pool = ThreadPoolExecutor(max_workers=10)

def create_jwt_token(user_id: int, username: str, role: str) -> str:
    payload = {
        'user_id': user_id,
        'username': username,
        'role': role,
        'exp': datetime.utcnow() + timedelta(hours=24)
    }
    return jwt.encode(payload, Config.SECRET_KEY, algorithm='HS256')

def verify_jwt_token(token: str) -> Optional[dict]:
    try:
        payload = jwt.decode(token, Config.SECRET_KEY, algorithms=['HS256'])
        return payload
    except jwt.ExpiredSignatureError:
        return None
    except jwt.InvalidTokenError:
        return None

def require_auth(roles: List[str] = None):
    def decorator(f):
        def decorated_function(*args, **kwargs):
            token = request.headers.get('Authorization', '').replace('Bearer ', '')
            if not token:
                return jsonify({'error': 'Authentication required'}), 401
            
            user_data = verify_jwt_token(token)
            if not user_data:
                return jsonify({'error': 'Invalid or expired token'}), 401
            
            if roles and user_data.get('role') not in roles:
                return jsonify({'error': 'Insufficient permissions'}), 403
            
            request.user = user_data
            return f(*args, **kwargs)
        return decorated_function
    return decorator

def cache_key():
    return f"cache:{request.path}:{hash(frozenset(request.args.items()))}"

@app.before_request
def before_request():
    request.start_time = time.time()

@app.after_request
def after_request(response):
    duration = time.time() - request.start_time
    REQUEST_DURATION.observe(duration)
    REQUEST_COUNT.labels(
        method=request.method,
        endpoint=request.endpoint,
        status=response.status_code
    ).inc()
    return response

class StudentManager:
    def __init__(self):
        self.predictor = PerformancePredictor()
        self.analytics_engine = None
    
    def get_db_session(self):
        return SessionLocal()
    
    def get_all_students(self, filters: Dict = None) -> List[Dict]:
        session = self.get_db_session()
        try:
            query = session.query(Student).filter(Student.is_active == True)
            
            if filters:
                if 'grade' in filters:
                    query = query.filter(Student.grade == filters['grade'])
                if 'min_marks' in filters:
                    query = query.filter(Student.marks >= filters['min_marks'])
                if 'max_marks' in filters:
                    query = query.filter(Student.marks <= filters['max_marks'])
                if 'search' in filters:
                    search = f"%{filters['search']}%"
                    query = query.filter(Student.name.ilike(search))
            
            students = query.all()
            return [self._student_to_dict(s) for s in students]
        finally:
            session.close()
    
    def create_student(self, student_data: Dict) -> Tuple[bool, str, Dict]:
        session = self.get_db_session()
        try:
            # Validate student data
            is_valid, message = self.validate_student_data(student_data)
            if not is_valid:
                return False, message, {}
            
            existing = session.query(Student).filter(Student.id == student_data['id']).first()
            if existing:
                return False, "Student ID already exists", {}
            
            student = Student(
                id=student_data['id'],
                name=student_data['name'],
                age=student_data['age'],
                grade=student_data['grade'],
                marks=student_data['marks'],
                email=student_data.get('email'),
                phone=student_data.get('phone'),
                address=student_data.get('address')
            )
            
            session.add(student)
            session.commit()
            
            cache.delete('students_all')
            
            return True, "Student created successfully", self._student_to_dict(student)
        except Exception as e:
            session.rollback()
            return False, f"Database error: {str(e)}", {}
        finally:
            session.close()
    
    def validate_student_data(self, data: Dict) -> Tuple[bool, str]:
        required_fields = ['id', 'name', 'age', 'grade', 'marks']
        for field in required_fields:
            if field not in data or not data[field]:
                return False, f"Missing required field: {field}"
        
        if not data['id'].isdigit():
            return False, "Student ID must be numeric"
        
        try:
            age = int(data['age'])
            if age < 5 or age > 100:
                return False, "Age must be between 5 and 100"
        except ValueError:
            return False, "Age must be a number"
        
        if data['grade'].upper() not in 'ABCDEF':
            return False, "Grade must be A-F"
        
        try:
            marks = int(data['marks'])
            if marks < 0 or marks > 100:
                return False, "Marks must be between 0 and 100"
        except ValueError:
            return False, "Marks must be a number"
        
        return True, "Valid"
    
    def _student_to_dict(self, student: Student) -> Dict:
        return {
            'id': student.id,
            'name': student.name,
            'age': student.age,
            'grade': student.grade,
            'marks': student.marks,
            'email': student.email,
            'phone': student.phone,
            'address': student.address,
            'created_at': student.created_at.isoformat(),
            'updated_at': student.updated_at.isoformat()
        }

student_manager = StudentManager()

@app.route('/api/v1/students', methods=['GET', 'POST'])
@require_auth(roles=['admin', 'teacher', 'user'])
@limiter.limit("100 per minute")
@cache.cached(timeout=60, key_prefix=cache_key)
def api_students():
    if request.method == 'GET':
        filters = {
            'grade': request.args.get('grade'),
            'min_marks': request.args.get('min_marks', type=int),
            'max_marks': request.args.get('max_marks', type=int),
            'search': request.args.get('search')
        }
        filters = {k: v for k, v in filters.items() if v is not None}
        
        students = student_manager.get_all_students(filters)
        return jsonify({
            'success': True,
            'data': students,
            'count': len(students),
            'filters_applied': filters
        })
    
    else:  # POST
        if request.user.get('role') not in ['admin', 'teacher']:
            return jsonify({'error': 'Insufficient permissions'}), 403
        
        data = request.get_json()
        success, message, student = student_manager.create_student(data)
            log_audit(
                user_id=request.user['user_id'],
                action='CREATE_STUDENT',
                resource=f"student:{student['id']}",
                details=f"Created student {student['name']}"
            )
            
            asyncio.run(send_notification_async(
                f"New student created: {student['name']} (ID: {student['id']})"
            ))
            
            return jsonify({
                'success': True,
                'message': message,
                'data': student
            }), 201
        else:
            return jsonify({
                'success': False,
                'error': message
            }), 400

@app.route('/api/v1/students/<student_id>', methods=['GET', 'PUT', 'DELETE'])
@require_auth(roles=['admin', 'teacher', 'user'])
def api_student(student_id):
    session = SessionLocal()
    try:
        student = session.query(Student).filter(Student.id == student_id, Student.is_active == True).first()
        
        if not student:
            return jsonify({'error': 'Student not found'}), 404
        
        if request.method == 'GET':
            return jsonify({
                'success': True,
                'data': student_manager._student_to_dict(student)
            })
        
        elif request.method == 'PUT':
            if request.user.get('role') not in ['admin', 'teacher']:
                return jsonify({'error': 'Insufficient permissions'}), 403
            
            data = request.get_json()
            
            for field in ['name', 'age', 'grade', 'marks', 'email', 'phone', 'address']:
                if field in data:
                    setattr(student, field, data[field])
            
            student.updated_at = datetime.utcnow()
            session.commit()
            
            cache.delete('students_all')
            log_audit(
                user_id=request.user['user_id'],
                action='UPDATE_STUDENT',
                resource=f"student:{student_id}",
                details="Updated student information"
            )
            
            return jsonify({
                'success': True,
                'message': 'Student updated successfully',
                'data': student_manager._student_to_dict(student)
            })
        
        elif request.method == 'DELETE':
            if request.user.get('role') != 'admin':
                return jsonify({'error': 'Admin access required'}), 403
            
            student.is_active = False
            session.commit()
            
            cache.delete('students_all')
            log_audit(
                user_id=request.user['user_id'],
                action='DELETE_STUDENT',
                resource=f"student:{student_id}",
                details="Soft deleted student"
            )
            
            return jsonify({
                'success': True,
                'message': 'Student deleted successfully'
            })
    
    finally:
        session.close()

@app.route('/api/v1/analytics/advanced', methods=['GET'])
@require_auth(roles=['admin', 'teacher'])
def advanced_analytics():
    students = student_manager.get_all_students()
    
    if not students:
        return jsonify({'error': 'No student data available'}), 404
    
    analytics_engine = AdvancedAnalytics(students)
    results = analytics_engine.perform_comprehensive_analysis()
    
    return jsonify({
        'success': True,
        'analytics': results,
        'generated_at': datetime.utcnow().isoformat()
    })

@app.route('/api/v1/predict/performance', methods=['POST'])
@require_auth(roles=['admin', 'teacher'])
def predict_performance():
    data = request.get_json()
    age = data.get('age')
    previous_marks = data.get('previous_marks', 0)
    
    if not age:
        return jsonify({'error': 'Age is required'}), 400
    
    predicted_marks = student_manager.predictor.predict(age, previous_marks)
    
    if predicted_marks is None:
        return jsonify({'error': 'Prediction model not trained or insufficient data'}), 400
    
    return jsonify({
        'success': True,
        'prediction': {
            'age': age,
            'previous_marks': previous_marks,
            'predicted_marks': round(predicted_marks, 2),
            'confidence': 'medium'  # This could be enhanced with prediction intervals
        }
    })

@app.route('/api/v1/export/students', methods=['GET'])
@require_auth(roles=['admin', 'teacher'])
def export_students():
    format_type = request.args.get('format', 'csv').lower()
    students = student_manager.get_all_students()
    
    if format_type == 'csv':
        output = io.StringIO()
        writer = csv.writer(output)
        writer.writerow(['ID', 'Name', 'Age', 'Grade', 'Marks', 'Email', 'Phone'])
        for student in students:
            writer.writerow([
                student['id'], student['name'], student['age'],
                student['grade'], student['marks'], student.get('email', ''),
                student.get('phone', '')
            ])
        output.seek(0)
        
        return send_file(
            io.BytesIO(output.getvalue().encode('utf-8')),
            mimetype='text/csv',
            as_attachment=True,
            download_name=f'students_export_{datetime.utcnow().strftime("%Y%m%d_%H%M%S")}.csv'
        )
    
    elif format_type == 'json':
        return jsonify({
            'success': True,
            'exported_at': datetime.utcnow().isoformat(),
            'count': len(students),
            'data': students
        })
    
    else:
        return jsonify({'error': 'Unsupported format. Use csv or json'}), 400

@app.route('/metrics')
def metrics():
    return generate_latest(), 200, {'Content-Type': CONTENT_TYPE_LATEST}

@app.route('/admin/dashboard')
@require_auth(roles=['admin'])
def admin_dashboard():
    session = SessionLocal()
    try:
        total_students = session.query(Student).filter(Student.is_active == True).count()
        recent_students = session.query(Student).order_by(Student.created_at.desc()).limit(5).all()
        
        cache_info = redis_client.info('memory') if redis_client else {}
        system_stats = {
            'total_students': total_students,
            'cache_usage': cache_info.get('used_memory_human', 'N/A'),
            'uptime': time.time() - app.start_time
        }
        
        return jsonify({
            'success': True,
            'dashboard': {
                'system_stats': system_stats,
                'recent_students': [student_manager._student_to_dict(s) for s in recent_students],
                'quick_actions': ['backup', 'analytics', 'reports', 'user_management']
            }
        })
    finally:
        session.close()

async def send_notification_async(message: str):
    """Send notification asynchronously"""
    # This could be extended to support multiple channels (email, SMS, push)
    print(f"Notification: {message}")  # Replace with actual notification service

def log_audit(user_id: int, action: str, resource: str, details: str = ""):
    """Enhanced audit logging"""
    session = SessionLocal()
    try:
        audit_log = AuditLog(
            user_id=user_id,
            action=action,
            resource=resource,
            details=details,
            ip_address=request.remote_addr,
            user_agent=request.headers.get('User-Agent')
        )
        session.add(audit_log)
        session.commit()
    finally:
        session.close()

def backup_to_s3():
    """Backup database to AWS S3"""
    if not s3_client:
        return
    
    try:
        # Create backup
        backup_data = json.dumps(student_manager.get_all_students())
        backup_key = f"backups/students_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}.json"
        
        s3_client.put_object(
            Bucket=Config.S3_BUCKET,
            Key=backup_key,
            Body=backup_data,
            ContentType='application/json'
        )
        
        log_audit(None, 'AUTO_BACKUP', 'system', f"Backup created: {backup_key}")
    except Exception as e:
        print(f"Backup failed: {e}")

def schedule_backup():
    """Schedule regular backups"""
    while True:
        time.sleep(Config.BACKUP_INTERVAL)
        thread_pool.submit(backup_to_s3)

if Config.ENABLE_K8S:
    try:
        config.load_incluster_config()  # For running inside cluster
    except:
        config.load_kube_config()  # For development
    
    k8s_apps_v1 = client.AppsV1Api()

@app.route('/api/v1/system/scale', methods=['POST'])
@require_auth(roles=['admin'])
def scale_application():
    if not Config.ENABLE_K8S:
        return jsonify({'error': 'Kubernetes integration disabled'}), 400
    
    data = request.get_json()
    replicas = data.get('replicas', 1)
        patch = {'spec': {'replicas': replicas}}
        k8s_apps_v1.patch_namespaced_deployment_scale(
            name='student-management',
            namespace='default',
            body=patch
        )
        
        return jsonify({
            'success': True,
            'message': f'Scaled application to {replicas} replicas'
        })
    except Exception as e:
        return jsonify({'error': f'Scaling failed: {str(e)}'}), 500

@app.before_request
def check_maintenance_mode():
    if redis_client.get('maintenance_mode') and not request.endpoint == 'metrics':
        return jsonify({
            'error': 'System under maintenance',
            'estimated_recovery': redis_client.get('maintenance_eta')
        }), 503

@app.route('/api/v1/system/maintenance', methods=['POST'])
@require_auth(roles=['admin'])
def toggle_maintenance():
    data = request.get_json()
    enable = data.get('enable', False)
    eta = data.get('eta', '30 minutes')
    
    if enable:
        redis_client.setex('maintenance_mode', 3600, 'true')  # 1 hour
        redis_client.setex('maintenance_eta', 3600, eta)
        message = 'Maintenance mode enabled'
    else:
        redis_client.delete('maintenance_mode', 'maintenance_eta')
        message = 'Maintenance mode disabled'
    
    return jsonify({'success': True, 'message': message})

@app.route('/api/v1/insights/recommendations', methods=['GET'])
@require_auth(roles=['admin', 'teacher'])
def get_ai_recommendations():
    students = student_manager.get_all_students()
    
    if not students:
        return jsonify({'error': 'No data for analysis'}), 400
    
    recommendations = []
    
    struggling = [s for s in students if s['marks'] < 50]
    if struggling:
        recommendations.append({
            'type': 'intervention',
            'priority': 'high',
            'message': f'{len(struggling)} students are struggling (marks < 50)',
            'students': [s['id'] for s in struggling]
        })
    
    high_performers = [s for s in students if s['marks'] >= 90]
    if high_performers:
        recommendations.append({
            'type': 'recognition',
            'priority': 'low',
            'message': f'{len(high_performers)} high performers identified',
            'students': [s['id'] for s in high_performers]
        })
    
    grade_counts = {}
    for student in students:
        grade_counts[student['grade']] = grade_counts.get(student['grade'], 0) + 1
    
    unbalanced_grades = [grade for grade, count in grade_counts.items() if count < len(students) * 0.1]
    if unbalanced_grades:
        recommendations.append({
            'type': 'distribution',
            'priority': 'medium',
            'message': f'Unbalanced grade distribution: {unbalanced_grades}'
        })
    
    return jsonify({
        'success': True,
        'recommendations': recommendations,
        'generated_at': datetime.utcnow().isoformat()
    })

@app.route('/api/v1/collaboration/notes', methods=['GET', 'POST'])
@require_auth(roles=['admin', 'teacher'])
def student_notes():
    student_id = request.args.get('student_id')
    
    if request.method == 'GET':
        notes = redis_client.get(f'student_notes:{student_id}') or '[]'
        return jsonify({
            'success': True,
            'notes': json.loads(notes)
        })
    
    else: 
        data = request.get_json()
        new_note = {
            'id': secrets.token_hex(8),
            'content': data['content'],
            'author': request.user['username'],
            'created_at': datetime.utcnow().isoformat()
        }
        
        existing_notes = redis_client.get(f'student_notes:{student_id}') or '[]'
        notes = json.loads(existing_notes)
        notes.append(new_note)
        
        redis_client.setex(
            f'student_notes:{student_id}',
            86400 * 30,  # 30 days
            json.dumps(notes)
        )
        
        return jsonify({
            'success': True,
            'note': new_note
        })

@app.route('/health')
def health_check():
    """Comprehensive health check endpoint"""
    health_status = {
        'status': 'healthy',
        'timestamp': datetime.utcnow().isoformat(),
        'checks': {}
    }
    
    try:
        session = SessionLocal()
        session.execute('SELECT 1')
        health_status['checks']['database'] = 'healthy'
        session.close()
    except Exception as e:
        health_status['checks']['database'] = f'unhealthy: {str(e)}'
        health_status['status'] = 'degraded'

    try:
        redis_client.ping()
        health_status['checks']['redis'] = 'healthy'
    except Exception as e:
        health_status['checks']['redis'] = f'unhealthy: {str(e)}'
        health_status['status'] = 'degraded'
    
    try:
        if not os.path.exists('temp'):
            os.makedirs('temp')
        with open('temp/healthcheck', 'w') as f:
            f.write('test')
        os.remove('temp/healthcheck')
        health_status['checks']['storage'] = 'healthy'
    except Exception as e:
        health_status['checks']['storage'] = f'unhealthy: {str(e)}'
        health_status['status'] = 'degraded'
    
    return jsonify(health_status)

def initialize_application():
    """Initialize application with advanced features"""
    app.start_time = time.time()
    
    threading.Thread(target=schedule_backup, daemon=True).start()
    
    students = student_manager.get_all_students()
    if len(students) >= 10:
        student_manager.predictor.train(students)

    cache.set('total_students', len(students))
    
    print("Advanced Student Management System initialized successfully")

if __name__ == '__main__':
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s %(levelname)s %(name)s %(message)s',
        handlers=[
            RotatingFileHandler('app.log', maxBytes=10000000, backupCount=5),
            logging.StreamHandler()
        ]
    )
    
    initialize_application()
    app.run(host='0.0.0.0', port=5000, debug=False)